# Import Libraries

In [54]:
# Import necessary libraries and functions
import pandas as pd
import numpy as np
from pathlib import Path

# Modeling + evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, roc_auc_score, average_precision_score,
    confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay
)
import matplotlib.pyplot as plt

# Set up helper functions and constants
* Defines constants and small helper functions that will be reused later when cleaning and transforming raw tables

In [55]:
# Sets reference date for calculating things like account age
DATA_CUTOFF = pd.to_datetime("2025-07-01")  # end of data window (per brief)

# Set random seed for reproducibility.
RANDOM_STATE = 42

# Safe wrapper around pd.read_csv
def safe_read(name):
    p = Path(name)
    if not p.exists():
        raise FileNotFoundError(f"Missing file: {name} (expected in {Path.cwd()})")
    return pd.read_csv(p)

# Helper for session aggregation
# Takes user’s first session and last session, and computes how many weeks they’ve been active
def weeks_between(min_ts, max_ts):
    if pd.isna(min_ts) or pd.isna(max_ts) or max_ts < min_ts:
        return np.nan
    return max(1.0, (max_ts - min_ts).days / 7.0)

# Read the raw CSVs and standardize data types
* Loads the raw data (users, sessions, events, billing)
* Standardizes date/time columns into a consistent datetime format
* Prepares the data for feature engineering

In [56]:
# Loads each dataset into a pandas DataFrame
users   = safe_read("users.csv") # safe_read() gives error message if file is missing
sessions= safe_read("sessions.csv")
events  = safe_read("events.csv")
billing = safe_read("billing.csv")

# Converts the signup_date column into a datetime object.
for col in ["signup_date"]:
    if col in users.columns:
        users[col] = pd.to_datetime(users[col], errors="coerce") # errors="coerce" means if a row has something invalid, doesn't crash

# Ensure session_start and session_end are proper datetimes
# Allows us to compute session length, first session date, and last session date per user
for col in ["session_start", "session_end"]:
    if col in sessions.columns:
        sessions[col] = pd.to_datetime(sessions[col], errors="coerce")

# Converts the event timestamp (ts) into datetime
if "ts" in events.columns:
    events["ts"] = pd.to_datetime(events["ts"], errors="coerce")

# Feature Engineering
* Turns raw logs → engineered features (numeric + categorical inputs).
* Organizes them into per-user rows (so every user has one feature vector)
* Creates the foundation for the churn prediction dataset

### Users
* Anchor” table — one row per user
* Contains demographics (plan_tier, company_size, region), acquisition channel and churn label (churned_30d, churned_90d)
* Gives who the user is and provides the target variable (Y)

In [60]:
# Choose 90-day churn horizon
label_col = "churned_90d" if "churned_90d" in users.columns else "churned_30d" # If 90 days not in users, choose 30 days
if label_col not in users.columns:
    raise ValueError("Neither churned_90d nor churned_30d exists in users.csv") # Error message

user_feats = users.copy()

# Calculate account age (days since signup to data cutoff)
if "signup_date" in user_feats.columns:
    user_feats["account_age_days"] = (DATA_CUTOFF - user_feats["signup_date"]).dt.days
    user_feats.drop(columns=["signup_date"], inplace=True, errors="ignore")

## Sessions
* Raw sessions logs → aggregated into behavioral metrics like sessions_per_week, avg_session_length, device_variety
* Captures how much the user engages with Atlassian products.

In [61]:
if len(sessions) > 0:
    ses = sessions.copy()
    # Safety: if session_length_sec missing, compute from start/end
    if "session_length_sec" not in ses.columns and {"session_start", "session_end"} <= set(ses.columns):
        ses["session_length_sec"] = (ses["session_end"] - ses["session_start"]).dt.total_seconds()

    # per-user aggregates
    ses_agg = ses.groupby("user_id").agg(
        sessions_count=("session_id", "nunique"),
        avg_session_length=("session_length_sec", "mean"),
        p75_session_length=("session_length_sec", lambda x: np.nanpercentile(x, 75)),
        first_session=("session_start", "min"),
        last_session=("session_end", "max"),
        device_variety=("device", pd.Series.nunique) if "device" in ses.columns else ("session_id", "count"),
        os_variety=("os", pd.Series.nunique) if "os" in ses.columns else ("session_id", "count"),
        app_version_variety=("app_version", pd.Series.nunique) if "app_version" in ses.columns else ("session_id", "count"),
        country_variety=("country", pd.Series.nunique) if "country" in ses.columns else ("session_id", "count")
    )

    # weeks active + sessions per week
    ses_agg["weeks_active"] = (ses_agg.apply(lambda r: weeks_between(r["first_session"], r["last_session"]), axis=1))
    ses_agg["sessions_per_week"] = ses_agg["sessions_count"] / ses_agg["weeks_active"]

    # Clean up infinities
    ses_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
else:
    ses_agg = pd.DataFrame(columns=["user_id"])  # empty (edge case)


## Events (feature adoption features)
* Product usage logs → aggregated into unique_features_used, success_rate, days_to_first_event.
* Captures what the user is doing inside the product and how effectively.

In [62]:
if len(events) > 0:
    ev = events.copy()
    ev_agg = ev.groupby("user_id").agg(
        total_events=("event_id", "nunique"),
        unique_features_used=("feature_name", pd.Series.nunique),
        success_rate=("success", "mean") if "success" in ev.columns else ("event_id", "count"),
        avg_latency_ms=("latency_ms", "mean") if "latency_ms" in ev.columns else ("event_id", "count"),
        first_event=("ts", "min"),
        last_event=("ts", "max")
    )

    # time to first feature use since signup (days)
    if "first_event" in ev_agg.columns and "signup_date" in users.columns:
        ev_agg = ev_agg.merge(users[["user_id", "signup_date"]], on="user_id", how="left")
        ev_agg["days_to_first_event"] = (ev_agg["first_event"] - ev_agg["signup_date"]).dt.days
        ev_agg.drop(columns=["signup_date"], inplace=True, errors="ignore")
else:
    ev_agg = pd.DataFrame(columns=["user_id"])

## Billing (financial health features)
* Invoices and payments → aggregated into avg_mrr, ever_overdue, total_tickets
* Captures the business relationship health (financial reliability, support burden)

In [63]:
# 2.4 Billing-level features (financial health)
if len(billing) > 0:
    bill = billing.copy()
    if "month" in bill.columns:
        # parse month if provided as string
        bill["month"] = pd.to_datetime(bill["month"], errors="coerce")

    bill_agg = bill.groupby("user_id").agg(
        avg_mrr=("mrr", "mean") if "mrr" in bill.columns else ("user_id", "size"),
        max_mrr=("mrr", "max") if "mrr" in bill.columns else ("user_id", "size"),
        min_mrr=("mrr", "min") if "mrr" in bill.columns else ("user_id", "size"),
        months_billed=("month", "nunique") if "month" in bill.columns else ("user_id", "size"),
        ever_overdue=("invoices_overdue", "max") if "invoices_overdue" in bill.columns else ("user_id", "size"),
        total_tickets=("support_ticket_count", "sum") if "support_ticket_count" in bill.columns else ("user_id", "size"),
        discount_ever=("discount_applied", "max") if "discount_applied" in bill.columns else ("user_id", "size"),
        active_seats_max=("active_seats", "max") if "active_seats" in bill.columns else ("user_id", "size"),
    )
else:
    bill_agg = pd.DataFrame(columns=["user_id"])

# Merging
* Feature engineering gets combined into a single dataset to feed into a machine learning model

In [64]:
df = user_feats.merge(ses_agg.reset_index(), on="user_id", how="left")
df = df.merge(ev_agg.reset_index(), on="user_id", how="left")
df = df.merge(bill_agg.reset_index(), on="user_id", how="left")

# Target
y = df[label_col].astype(int)

# Drop identifiers / leakage columns
drop_cols = ["user_id", label_col, "first_session", "last_session", "first_event", "last_event"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

# Train / Test split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
)

# Preprocess
* Prepare features (X) so they’re in a form that machine learning models can use

In [70]:
# Splits features into categorical (e.g., plan_tier, region) and numeric (e.g., avg_session_length, account_age_days)
# Models like logistic regression can’t handle raw strings → we need to encode them
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_train.select_dtypes(include=["number", "bool"]).columns.tolist()

# Imputer: fills in missing values with the median (robust against outliers)
# Scaler: normalizes numeric features (so values like 1000 MRR and 0.3 success rate are on comparable scales)
numeric_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))  # with_mean False to be safe with sparse concat
])

# Imputer: fills missing categories with the most common one.
# OneHotEncoder: turns categories into binary columns.
# Example: plan_tier = Premium → [Free=0, Standard=0, Premium=1, Enterprise=0].
# handle_unknown="ignore": prevents errors if a new category shows up at test time.
categorical_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

# Combines the numeric and categorical pipelines.
# Applies each transformation to the right columns.
# Now, every row becomes a fully numeric feature vector ready for modeling.
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ],
    remainder="drop"
)